In [1]:
!pip install --upgrade jinja2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.2/133.2 kB 2.9 MB/s eta 0:00:00
  Attempting uninstall: jinja2
    Found existing installation: Jinja2 3.1.2
    Uninstalling Jinja2-3.1.2:
      Successfully uninstalled Jinja2-3.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.


In [ ]:
!pip install git+https://github.com/huggingface/transformers.git

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-9gz72peq
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-9gz72peq
  Resolved https://github.com/huggingface/transformers.git to commit 2382706a1ca038099214664dfa40f2f06883072c


In [ ]:
import torch
from transformers import pipeline


In [ ]:
!pip install gradio_client

In [ ]:
!pip install jinja2==3.1.3

In [ ]:
!pip install pyecharts

In [ ]:
import json
#from dotenv import load_dotenv
from datetime import datetime, timedelta
#import openai
import os
from gradio_client import Client
from pyecharts import *
#from plot import plot_line_data_to_html, plot_accident_data_to_html, plot_gpt_data_to_html
#PLOT GPT generated top station data
import re
import matplotlib.pyplot as plt
from pyecharts.charts import Bar, Page, Pie, Grid
from pyecharts import options as opts


#load_dotenv()

# Define the function to plot the gpt data
def plot_gpt_data_to_html(filtered_gpt_data):
    data = filtered_gpt_data

    # Updated pattern to handle both data formats
    pattern = r'(\w[\w\s,]*[^\s])\s\((\d+)\)|\(([\w\s,]+), (\d+)\)'
    matches = re.findall(pattern, data)

    # Extracting station names and frequencies considering both matching groups from the updated pattern
    stations = [match[0] if match[2] == '' else match[2] for match in matches]
    frequencies = [int(match[1]) if match[3] == '' else int(match[3]) for match in matches]

    bar_chart = (
        Bar()
        .add_xaxis(stations)
        .add_yaxis("Frequenz", frequencies, tooltip_opts=opts.TooltipOpts(is_show=True, trigger="axis", axis_pointer_type="cross"))
        .set_global_opts(
            title_opts=opts.TitleOpts(title="Top 10 Betroffenen Stationen"),
            xaxis_opts=opts.AxisOpts(
                name="Stationen",  # X-Axis title
                axislabel_opts=opts.LabelOpts(
                    rotate=45,   # Try 45-degree rotation
                    font_size=8  # Further reduce font size
                )
            ),
            yaxis_opts=opts.AxisOpts(
                name="Frequenz",  # Y-Axis title
            )
        )
    )

    return bar_chart.render_notebook()

# Configure the OpenAI API client
#openai.api_type = os.getenv("OPENAI_API_TYPE")
#openai.api_version = os.getenv("OPENAI_API_VERSION")
#openai.api_base = os.getenv("OPENAI_API_BASE")  # Your Azure OpenAI resource's endpoint value.
#openai.api_key = os.getenv("OPENAI_API_KEY")

# Define the function to generate the response
#def generate_response(resulted_data):
#    response = openai.ChatCompletion.create(
#        model="gpt-4",
        #engine=os.getenv("DEPLOYMENT_NAME"), # The deployment name you chose when you deployed the GPT-35-Turbo or GPT-4 model.
#        temperature = 0.4,
#        messages=[
#            {"role": "system", "content": "Du bist ein Analyse-Chatbot. Aus den bereitgestellten JSON-Daten antwortest du auf Nutzerfragen, um Statistiken basierend auf Benutzereingaben zu erstellen. Dies sind die Kontext-JSON-Daten:"+ resulted_data},
#            {"role": "user", "content": "Im JSON-Datenkontext der Wiener-Linie sind unter Titel betroffene Linien und unter 'Beschreibung' betroffene Stationen verzeichnet. Welche 10 Stationen sind am häufigsten betroffen? Geben Sie nur in diesem Format aus: (Stationsname, Gesamtzahl der Vorfälle). Zum Beispiel: (Rotkreuzplatz, 10)."
#            }
#        ]
#    )
#    return response['choices'][0]['message']['content']

def generate_response(resulted_data):


    response = openai.ChatCompletion.create(
    model="gpt-4-0314",
    # engine=os.getenv("DEPLOYMENT_NAME"), # The deployment name you chose when you deployed the GPT-35-Turbo or GPT-4 model.
    temperature=0.7,
    messages=[
      {"role": "user",
       "content": "You are an analyst. From the data provided, you answer user questions to create statistics based on user input. This is the context list data:" + resulted_data + "In the Vienna Line data context, affected lines are under title and under 'Description' lists affected stations. Which 10 stations are most frequently affected? Only output in this format: (station name, total number of incidents). For example: (Rotkreuzplatz, 10)."
      }
    ]
  )
    return response['choices'][0]['message']['content']



def generate_response_tinyllama(resulted_data):
    client = Client("https://tinyllama-tinyllama-chat.hf.space/--replicas/q15sq/")
    result = client.predict("You are an analyst. From the data provided, you answer user questions to create statistics based on user input. This is the context list data:" + resulted_data + "In the Vienna Line data context, affected lines are under title and under 'Description' lists affected stations. Which 10 stations are most frequently affected? Only output in this format: (station name, total number of incidents). For example: (Rotkreuzplatz, 10).",	 # str  in 'Message' Textbox component
                            api_name="/chat")

    return result

def generate_response_tinyllama2(resulted_data, temp):

    pipe = pipeline("text-generation", model="TinyLlama/TinyLlama-1.1B-Chat-v1.0", torch_dtype=torch.bfloat16)
    # We use the tokenizer's chat template to format each message - see https://huggingface.co/docs/transformers/main/en/chat_templating
    messages = [
    {
        "role": "system",
        "content": "You are an analyst. From the data provided, you answer user questions to create statistics based on user input. This is the context list data:" + resulted_data + "In the Vienna Line data context, affected lines are under title and under 'Description' lists affected stations. Which 10 stations are most frequently affected? Only output in this format: (station name, total number of incidents). For example: (Rotkreuzplatz, 10)."
    }
    ]
    prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    outputs = pipe(prompt, max_new_tokens=512, do_sample=True, temperature=temp, top_k=50, top_p=0.95)

    #print(outputs[0]["generated_text"])

    #client = Client("https://tinyllama-tinyllama-chat.hf.space/--replicas/q15sq/")
    #result = client.predict("You are an analyst. From the data provided, you answer user questions to create statistics based on user input. This is the context list data:" + resulted_data + "In the Vienna Line data context, affected lines are under title and under 'Description' lists affected stations. Which 10 stations are most frequently affected? Only output in this format: (station name, total number of incidents). For example: (Rotkreuzplatz, 10).",	 # str  in 'Message' Textbox component
    #                        api_name="/chat")

    return outputs[0]["generated_text"]


# Define the function to filter the accident data
def get_incident_data(data, start_time, end_time, temp):
    # Convert the provided timestamp range strings to datetime objects
    date_format = "%Y-%m-%d %H:%M:%S"
    start_datetime = datetime.strptime(start_time, date_format)
    end_datetime = datetime.strptime(end_time, date_format)

    #gpt filter
    filtered_incidents = [
        {
            'title': entry['title'],
            'description': entry['description']
        }
        for entry in data['incidents']
        if start_datetime <= datetime.strptime(entry['start'], date_format) <= end_datetime
    ]

    resulted_data = json.dumps(filtered_incidents)

    print(resulted_data)

    #filtered_gpt_data = ''
    filtered_gpt_data = generate_response_tinyllama2(resulted_data, temp)

    return filtered_gpt_data

# Define the function to generate the date ranges
def get_date_ranges():
    # Initialize the end date to today's date at 00:00:00
    end_time = datetime.strptime('2014-08-16 00:00:00.00000','%Y-%m-%d %H:%M:%S.%f')
    # Define the time interval (1 week = 7 days)
    interval = timedelta(days=1)
    date_ranges = []
    # Generate the ranges
    for _ in range(3):
        start_time = end_time - interval
        date_ranges.append((start_time, end_time))
        end_time = start_time
    return date_ranges


# Define the function to generate the Subway_line data
def get_line_data(date_ranges):
    with open('original_2014-8-15.json', 'r') as file:
        data = json.load(file)

    #top 10 line data
    lines_data_list = []
    for start, end in date_ranges:
        start_str = start.strftime('%Y-%m-%d %H:%M:%S')
        end_str = end.strftime('%Y-%m-%d %H:%M:%S')
        filetered_top_10_line_data = get_top10_lines(data, start_str, end_str)
        #print("Line data from data analysis:")
        lines_data_list.append(filetered_top_10_line_data)
    print(lines_data_list)
    return plot_line_data_to_html(lines_data_list)


# Define the function to generate the accident data
def get_accident_data(date_ranges):
    with open('original_2014-8-15.json', 'r') as file:
        data = json.load(file)

    #accident time sum daata
    accident_data_list = []
    for start, end in date_ranges:
        start_str = start.strftime('%Y-%m-%d %H:%M:%S')
        end_str = end.strftime('%Y-%m-%d %H:%M:%S')
        filtered_accident_time_data = get_accident_time_sum(data, start_str, end_str)
        #print("Filtered Accident time data")
        accident_data_list.append(filtered_accident_time_data)
    print(accident_data_list)
    return plot_accident_data_to_html(accident_data_list)


# Define the function to generate the gpt data
def get_gpt_data(date_ranges, temp):
    # Use the list of tuples in a for loop
    data = read_data()
    for start, end in date_ranges:
        start_str = start.strftime('%Y-%m-%d %H:%M:%S')
        end_str = end.strftime('%Y-%m-%d %H:%M:%S')

        print(f"Start: {start_str}, End: {end_str}")
        filtered_gpt_data = get_incident_data(data, start_str, end_str, temp)

        print("Filtered TinyLLAM station data:")
        print(filtered_gpt_data)
        break
    return plot_gpt_data_to_html(filtered_gpt_data)

# Define the function to read the data
def read_data():
    with open('original_2014-8-15.json', 'r') as file:
        data = json.load(file)
    return data
#get_line_data(get_date_ranges())
#get_accident_data(get_date_ranges())
#get_gpt_data(get_date_ranges())


In [ ]:
get_gpt_data(get_date_ranges(),0.7)

In [ ]:
get_gpt_data(get_date_ranges(),0.5)

In [ ]:
get_gpt_data(get_date_ranges(),0.4)

In [ ]:
get_gpt_data(get_date_ranges(),0.3)

In [ ]:
get_gpt_data(get_date_ranges(),0.2)

In [ ]:
get_gpt_data(get_date_ranges(),0.1)

In [ ]:
get_gpt_data(get_date_ranges(),0.8)

In [ ]:
get_gpt_data(get_date_ranges(),0.9)